In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("../csv/AAPL.csv")

In [ ]:
df.head()

In [ ]:
# 컬럼의 데이터를 인덱스로 변경
# set_index(컬럼명)
df = df.set_index('Date')

In [ ]:
# 컬럼의 이름을 변경하려면?
df.columns = ['시가', '고가', '저가', '종가', '수정종가', '거래량']

In [ ]:
df.head(1)

In [ ]:
df.index.name = "날짜"

In [ ]:
df.head(1)

In [ ]:
# 데이터프레임에서 특정 컬럼으로 필터
df.loc[:,'시가' : '수정종가']
df.iloc[:, 1:3]

In [ ]:
# 시가가 100달러 이상인 경우
df.loc[df['시가'] >= 100]

In [ ]:
# query() 함수
# query(조건식)
df.query("시가 >= 100 & 고가 < 150")

In [ ]:
data = [
    [100, 900, 800, 700], 
    [1200, 1400, 900, 800]
]
columns = [
    ['영업이익', '영업이익', '당기순이익', '당기순이익'],
    ['컨센서스', '잠정치', '컨센서스', '잠정치']
]

df = pd.DataFrame(data, columns=columns)

In [ ]:
df

In [ ]:
# stack(), unstack()
stack_df = df.stack()

In [ ]:
stack_df

In [ ]:
stack_df.stack()

In [ ]:
stack_df.unstack()

In [ ]:
df

In [ ]:
df.stack(level=0)

## 할로윈 투자 전략
1. 11월 1일에 구매 -> 6개월간 유지 4월 말일에 판매
2. 11월 첫날의 시가로 구매
3. 4월 마지막 날의 종가 판매
4. 수익율 = (3번과정 - 2번 과정) / 2번 과정
5. AMZN 데이터로 2000년부터 2019년까지 데이터로 백테스팅

In [ ]:
df = pd.read_csv("../csv/AMZN.csv")

In [ ]:
df

In [ ]:
# Date 컬럼의 데이터를 시계열로 변경
df['Date'] = pd.to_datetime(df['Date'], format="%Y-%m-%d")

In [ ]:
# Date 컬럼을 인덱스로 변환
df.set_index('Date', inplace=True)

In [ ]:
df.loc["2000-11"].iloc[0]['Open']

In [ ]:
df.loc["2001-04"].iloc[-1]['Close']

In [ ]:
for i in range(2000, 2020, 1):
    buy_mon = str(i)+"-11"
    sell_mon = str(i+1)+"-04"
    print(buy_mon, sell_mon)

In [ ]:
acc_rtn = 1

for i in range(2000, 2019, 1):
    buy_mon = str(i) + "-11"
    sell_mon = str(i+1) + "-04"

    # 구입가 저장 
    buy = df.loc[buy_mon].iloc[0]['Open']
    sell = df.loc[sell_mon].iloc[-1]['Close']

    # 수익율 
    rtn = (sell - buy) / buy + 1

    # 누적 수익율
    acc_rtn *= rtn



In [ ]:
acc_rtn

In [19]:
# buyandhold 인 경우
buy = df.loc["2000-11"].iloc[0]['Open']
sell = df.loc['2019-04'].iloc[-1]['Close']

rtn = (sell-buy)/buy +1

rtn

53.794625340314134

In [20]:
# CAGR(연복리수익율)
# (수익율 ** (1/투자기간)) - 1 

#  할로윈 CAGR
CAGR = (acc_rtn ** (1/20)) - 1
print(CAGR * 100)

11.124213625243007


In [21]:
# 바이앤홀드 CAGR
CAGR = (rtn ** (1/20)) - 1 
print(CAGR * 100)

22.04976410456394


In [22]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [23]:
start = datetime(year = 2000, month=11, day=1)
start

datetime.datetime(2000, 11, 1, 0, 0)

In [24]:
end = start + relativedelta(months=5)
end

datetime.datetime(2001, 4, 1, 0, 0)

In [25]:
print(start.strftime("%Y-%m"), end.strftime("%Y-%m"))

2000-11 2001-04


In [41]:
def six_month(df, start = 2000, end = 2019, month=11):
    if 'Date' in df.columns:
        df['Date'] = pd.to_datetime(df['Date'], format="%Y-%m-%d")
        df.set_index('Date', inplace=True)

    # 누적 수익율 변수를 생성
    acc_rtn = 1

    for i in range(start, end):
        start_ = datetime(year=i, month=month, day=1)
        end_ = start_ + relativedelta(months=5)

        buy_mon = start_.strftime("%Y-%m")
        sell_mon = end_.strftime("%Y-%m")

        buy = df.loc[buy_mon].iloc[0]['Open']
        sell = df.loc[sell_mon].iloc[-1]['Close']

        rtn = (sell - buy) / buy + 1

        acc_rtn *= rtn
    
    return acc_rtn

In [42]:
data = pd.read_csv('../csv/AMZN.csv')

In [43]:
six_month(data)

8.244683980501206

In [44]:
for i in range(1, 13, 1):
    print(f"{i}월 :", six_month(data, month=i))

1월 : 4.470910943811504
2월 : 4.408415564523263
3월 : 18.50624959809526
4월 : 13.582616833702685
5월 : 4.304179383672146
6월 : 5.111566986774566
7월 : 3.589575312504607
8월 : 6.418767529470167
9월 : 1.3058661611037958
10월 : 1.9960729229032756
11월 : 8.244683980501206
12월 : 6.23900557812782
